# Classifier
For the first stage of this project, using trained data from nexto 2v2 using our observation builders, we create a model to predict what Nexto would do given our inputs. This is essentially model distillation.

In [1]:
import pickle
import os
from tqdm.notebook import tqdm
dir = "data"

In [2]:
def loadPKL(name):
    with open(os.path.join(dir, name), 'rb') as f:
        return [(x.astype('float32'), y.astype('float32'))for x,y in pickle.load(f)]
def loadAll():
    data = []
    files = [x for x in os.listdir(dir) if not x.startswith('.')]
    for f in tqdm(files):
        data+=loadPKL(f)
    return data

# Pytorch section

### Dataset

In [7]:
import numpy as np
import torch


from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from torch import Tensor
from torch.nn import Linear
from torch.nn import SELU
from torch.nn import Tanh
from torch.nn import Module

from torch.nn import MSELoss
from torch.optim import Adam

In [8]:
class NextoDataset(Dataset):
    def __init__(self):
        data = loadAll()
        self.X = []
        self.Y = []
        for x,y in data:
            self.X.append(x)
            self.Y.append(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        return [self.X[i], self.Y[i]]
    def get_splits(self, n_test = .2):
        test_size = round(n_test*len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

### Model

In [9]:
class Net(Module):
    # define model elements
    def __init__(self):
        super().__init__()
        # input to first hidden layer
        n_inputs = 119
        
        self.hidden1 = Linear(n_inputs, 256)
        self.act1 = SELU()

        self.hidden2 = Linear(256, 256)
        self.act2 = SELU()

        self.hidden3 = Linear(256, 128)
        self.act3 = SELU()

        self.hidden4 = Linear(128, 64)
        self.act4 = SELU()
        
        # third hidden layer and output
        self.hidden5 = Linear(64, 8)
        self.act5 = Tanh()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        
        X = self.hidden4(X)
        X = self.act4(X)
        
        X = self.hidden5(X)
        X = self.act5(X)
        return X

In [10]:
Net()

Net(
  (hidden1): Linear(in_features=119, out_features=256, bias=True)
  (act1): SELU()
  (hidden2): Linear(in_features=256, out_features=256, bias=True)
  (act2): SELU()
  (hidden3): Linear(in_features=256, out_features=128, bias=True)
  (act3): SELU()
  (hidden4): Linear(in_features=128, out_features=64, bias=True)
  (act4): SELU()
  (hidden5): Linear(in_features=64, out_features=8, bias=True)
  (act5): Tanh()
)

### Utility functions

In [11]:
def prepare_data(batch_size):
    # load the dataset
    dataset = NextoDataset()
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
    return train_dl, test_dl

In [12]:
def train_model(train_dl, valid_dl, model,epochs = 10):
    if torch.cuda.is_available():
        model = model.cuda()
    min_valid_loss = np.inf
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters())
    with tqdm(total=len(train_dl)) as bar:
        for e in range(epochs):
            train_loss = 0.0
            model.train()     # Optional when not using Model Specific layer
#             bar.reset()
            for data, labels in train_dl:
#                 bar.update(1)
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                optimizer.zero_grad()
                target = model(data)
                loss = criterion(target,labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            valid_loss = 0.0
            model.eval()     # Optional when not using Model Specific layer
            for data, labels in valid_dl:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target = model(data)
                loss = criterion(target,labels)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {e+1} \tTraining Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(valid_dl)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                torch.save(model.state_dict(), f'models/{valid_loss/len(valid_dl)*1000:.6f}.pth')


In [13]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# Training Loop

In [15]:
train_dl, test_dl = prepare_data(batch_size=128)
print(len(train_dl.dataset), len(test_dl.dataset))

  0%|          | 0/323 [00:00<?, ?it/s]

258658 64665


In [24]:
model = Net()
paths = [float(x[:-4]) for x in os.listdir('models') if not x.startswith('.')]
best = min(paths)

model.load_state_dict(torch.load(f'models/{best:.6f}.pth'))

<All keys matched successfully>

In [25]:
model

Net(
  (hidden1): Linear(in_features=119, out_features=256, bias=True)
  (act1): SELU()
  (hidden2): Linear(in_features=256, out_features=256, bias=True)
  (act2): SELU()
  (hidden3): Linear(in_features=256, out_features=128, bias=True)
  (act3): SELU()
  (hidden4): Linear(in_features=128, out_features=64, bias=True)
  (act4): SELU()
  (hidden5): Linear(in_features=64, out_features=8, bias=True)
  (act5): Tanh()
)

In [ ]:
# train the model
train_model(train_dl, test_dl, model, epochs = 10000)

  0%|          | 0/2021 [00:00<?, ?it/s]

Epoch 1 	Training Loss: 0.21746713860178868 	 Validation Loss: 0.010990423588413494
Validation Loss Decreased(inf--->5.561154) 	 Saving The Model
Epoch 2 	Training Loss: 0.2147982468527181 	 Validation Loss: 0.010320866708698952
Validation Loss Decreased(5.561154--->5.222359) 	 Saving The Model
Epoch 3 	Training Loss: 0.2132507605991524 	 Validation Loss: 0.010760503235777376
Epoch 4 	Training Loss: 0.21213402352464017 	 Validation Loss: 0.010422770331499605
Epoch 5 	Training Loss: 0.21108748401408736 	 Validation Loss: 0.012355612907485058
Epoch 6 	Training Loss: 0.21035372922616333 	 Validation Loss: 0.012735367233574154
Epoch 7 	Training Loss: 0.2095684059874838 	 Validation Loss: 0.011189185495197537
Epoch 8 	Training Loss: 0.20874481078629917 	 Validation Loss: 0.011515888300808992
Epoch 9 	Training Loss: 0.20815816730305087 	 Validation Loss: 0.010849783572519249
Epoch 10 	Training Loss: 0.20767590534061797 	 Validation Loss: 0.012090942811353405
Epoch 11 	Training Loss: 0.207112

Epoch 97 	Training Loss: 0.19167832935110052 	 Validation Loss: 0.012708246236733297
Epoch 98 	Training Loss: 0.19164787298496969 	 Validation Loss: 0.014355406165122986
Epoch 99 	Training Loss: 0.19143307685321184 	 Validation Loss: 0.013229090411201296
Epoch 100 	Training Loss: 0.19149995966455477 	 Validation Loss: 0.013216069564517778
Epoch 101 	Training Loss: 0.1912779656968336 	 Validation Loss: 0.012732978924932217
Epoch 102 	Training Loss: 0.19127210435715128 	 Validation Loss: 0.012425925185086699
Epoch 103 	Training Loss: 0.19117935178806025 	 Validation Loss: 0.013622875682450094
Epoch 104 	Training Loss: 0.19098058637122833 	 Validation Loss: 0.013623606016042204
Epoch 105 	Training Loss: 0.19107239544096705 	 Validation Loss: 0.013203134119746242
Epoch 106 	Training Loss: 0.19095091123589192 	 Validation Loss: 0.01196420416530413
Epoch 107 	Training Loss: 0.19085100301269964 	 Validation Loss: 0.012595710075891066
Epoch 108 	Training Loss: 0.19073817883016803 	 Validation 

In [ ]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))